In [1]:
#encoding: utf-8
import numpy as np
import pandas as pd
import xgboost as xgb

from dateutil import parser
from datetime import datetime
import math
data_path = "../data/"
result_path =  "../data/result/"
feature_path =  "../data/feature/"
analysis_path =  "../data/analysis/"
function_path =  "../data/function/"

# reading data
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'evaluation_public.csv')
entbase = pd.read_csv(data_path + '1entbase.csv')
alter = pd.read_csv(data_path + '2alter.csv')
branch = pd.read_csv(data_path + '3branch.csv')
invest = pd.read_csv(data_path + '4invest.csv')
right = pd.read_csv(data_path + '5right.csv')
project = pd.read_csv(data_path + '6project.csv')
lawsuit = pd.read_csv(data_path + '7lawsuit.csv')
breakfaith = pd.read_csv(data_path + '8breakfaith.csv')
recruit = pd.read_csv(data_path + '9recruit.csv')
qualification = pd.read_csv(data_path + '10qualification.csv')


/opt/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import math
def translateYear(date):
    if type(date) == float:
        return np.nan
    year = int(date[:4])
    month = int(date[-2:])
    return (year-2015)*12 + month

def translateYearALTER(date):
    if type(date) == float:
        return np.nan
    year = int(date[:4])
    month = int(date[-2:])
    return (year-2013)*12 + month

def translateYearRECRUIT(date):
    if type(date) == float:
        return np.nan
    year = int(date[:4])
    month = int(date[-2:])
    return (year-2008)*12 + month

def translateYearFB(date):
    if type(date) == float:
        return np.nan
    year = int(date[:4])
    month = int(date[5:-2])
    return (year-2010)*12 + month

import math
def translateEnddate(date):
    if type(date) == float:
        return 2015
    year = int(date[:4])
    return year

def get_HandE(new_right, col_name, prefix_index):
    right_type_HY_sta = new_right.groupby(['HY'])[col_name].agg({sum, max,min, np.mean}).add_prefix(col_name[: prefix_index] + '_HY_').reset_index()
    right_type_ETYPE_sta = new_right.groupby(['ETYPE'])[col_name].agg({sum, max,min, np.mean}).add_prefix(col_name[: prefix_index] + '_ETYPE_').reset_index()
    right_type_HYandETYPE_sta = new_right.groupby(['ETYPE', 'HY'])[col_name].agg({sum, max,min, np.mean}).add_prefix(col_name[: prefix_index] + '_HYandETYPE_').reset_index()
    
    new_right = pd.merge(new_right, right_type_HY_sta, on = 'HY', how = 'left')
    new_right = pd.merge(new_right, right_type_ETYPE_sta, on = 'ETYPE', how = 'left')
    new_right = pd.merge(new_right, right_type_HYandETYPE_sta, on = ['ETYPE', 'HY'], how = 'left') 
    
    new_right[col_name[:prefix_index] + '_HY_gap'] = new_right[col_name] - new_right[col_name[:prefix_index] + '_HY_mean'] 
    new_right[col_name[:prefix_index] + '_ETYPE_gap'] = new_right[col_name] - new_right[col_name[:prefix_index] + '_ETYPE_mean'] 
    new_right[col_name[:prefix_index] + '_HYandETYPE_gap'] = new_right[col_name] - new_right[col_name[:prefix_index] + '_HYandETYPE_mean']  
    
    return new_right

def get_HandEforEntbase(new_right, col_name, prefix_index):
    right_type_HY_sta = new_right.groupby(['HY'])[col_name].agg({max, np.mean, np.median}).add_prefix(col_name[: prefix_index] + '_HY_').reset_index()
    right_type_ETYPE_sta = new_right.groupby(['ETYPE'])[col_name].agg({ max, np.mean, np.median}).add_prefix(col_name[: prefix_index] + '_ETYPE_').reset_index()
    right_type_HYandETYPE_sta = new_right.groupby(['ETYPE', 'HY'])[col_name].agg({max, np.mean, np.median}).add_prefix(col_name[: prefix_index] + '_HYandETYPE_').reset_index()
    
    new_right = pd.merge(new_right, right_type_HY_sta, on = 'HY', how = 'left')
    new_right = pd.merge(new_right, right_type_ETYPE_sta, on = 'ETYPE', how = 'left')
    new_right = pd.merge(new_right, right_type_HYandETYPE_sta, on = ['ETYPE', 'HY'], how = 'left') 
    
    new_right[col_name[:prefix_index] + '_HY_gap'] = new_right[col_name] - new_right[col_name[:prefix_index] + '_HY_mean'] 
    new_right[col_name[:prefix_index] + '_ETYPE_gap'] = new_right[col_name] - new_right[col_name[:prefix_index] + '_ETYPE_mean'] 
    new_right[col_name[:prefix_index] + '_HYandETYPE_gap'] = new_right[col_name] - new_right[col_name[:prefix_index] + '_HYandETYPE_mean']  
    
    return new_right

def get_HandEandPforEntbase(df, col_name, prefix_index):
    df_PROV_sta = df.groupby(['PROV'])[col_name].agg({ max,np.mean, np.median}).add_prefix(col_name[: prefix_index]+ '_PROV_').reset_index()
    df_PROV_HY_sta = df.groupby(['PROV', 'HY'])[col_name].agg({ max,np.mean, np.median}).add_prefix(col_name[: prefix_index]+ '_PROVandHY_').reset_index()
    df_PROV_ETYPE_sta = df.groupby(['PROV', 'ETYPE'])[col_name].agg({max,  np.mean, np.median}).add_prefix(col_name[: prefix_index]+ '_PROVandETYPE_').reset_index()
    df_PROV_HY_ETYPE_sta = df.groupby(['PROV', 'HY', 'ETYPE'])[col_name].agg({max,np.mean, np.median}).add_prefix(col_name[: prefix_index]+ '_PROVandHYandETYPE_').reset_index()

    df = pd.merge(df, df_PROV_sta, on = 'PROV', how = 'left')
    df = pd.merge(df, df_PROV_HY_sta, on = ['PROV', 'HY'], how = 'left')
    df = pd.merge(df, df_PROV_ETYPE_sta, on = ['PROV', 'ETYPE'], how = 'left') 
    df = pd.merge(df, df_PROV_HY_ETYPE_sta, on = ['PROV', 'ETYPE', 'HY'], how = 'left') 
    
    
    df[col_name[:prefix_index] + '_PROV_gap'] = df[col_name] - df[col_name[:prefix_index] + '_PROV_mean'] 
    df[col_name[:prefix_index] + '_PROVandHY_gap'] = df[col_name] - df[col_name[:prefix_index] + '_PROVandHY_mean'] 
    df[col_name[:prefix_index] + '_PROVandETYPE_gap'] = df[col_name] - df[col_name[:prefix_index] + '_PROVandETYPE_mean']  
    df[col_name[:prefix_index] + '_PROVandHYandETYPE_gap'] = df[col_name] - df[col_name[:prefix_index] + '_PROVandHYandETYPE_mean']  

    return df    

# Train

In [3]:
train = pd.read_csv(data_path + 'train.csv')
train['ENDDATE_n'] = train.ENDDATE.apply(translateEnddate)
new_train = train.drop(['ENDDATE', 'ENDDATE_n'], axis = 1)

In [4]:
entbase[entbase.ZCZB.isnull()]

,PROV,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,FSTINUM,TZINUM
205866,12,s179317,1997,24.0,NaN,5,1.0,2.0,2.0,20.00,0.0,NaN
208414,12,s513597,1987,61.0,NaN,5,5.0,5.0,5.0,442523.23,62.0,NaN
253977,12,s241292,1996,65.0,NaN,5,NaN,1.0,0.0,NaN,0.0,NaN
394503,12,s140281,2011,52.0,NaN,8,NaN,1.0,0.0,NaN,0.0,NaN
396685,12,s299404,2012,72.0,NaN,5,NaN,1.0,1.0,NaN,0.0,NaN


In [5]:
len(entbase.EID.unique())

436798

In [6]:
len(alter.EID.unique())

136034

In [7]:
len(branch.EID.unique())

27736

In [8]:
len(invest.EID.unique())

30728

In [9]:
len(right.EID.unique())

120662

In [10]:
len(project.EID.unique())

8067

In [11]:
len(lawsuit.EID.unique())

10148

In [12]:
len(breakfaith.EID.unique())

1451

In [13]:
len(recruit.EID.unique())

41877

In [14]:
entbase.count()

PROV       436798
EID        436798
RGYEAR     436798
HY         436790
ZCZB       436793
ETYPE      436798
MPNUM      321207
INUM       433024
ENUM       433024
FINZB       34143
FSTINUM    431027
TZINUM       5448
dtype: int64

# Entbase Feature

In [15]:
entbase = pd.read_csv(data_path + '1entbase.csv')

In [16]:
entbase

,PROV,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,FSTINUM,TZINUM
0,11,p13836,1980,51.0,480.0,13,25.0,1.0,1.0,98041.0000,17.0,NaN
1,11,p49137,1993,74.0,30.0,13,0.0,1.0,1.0,570.0000,3.0,NaN
2,11,p108756,2015,69.0,1000.0,6,2.0,1.0,1.0,1000.0000,0.0,NaN
3,11,p7320,2005,74.0,240.0,5,0.0,2.0,2.0,1500.0000,0.0,NaN
4,11,p194070,2014,51.0,100000.0,3,3.0,2.0,2.0,1000.0000,7.0,NaN
5,11,p120116,2009,51.0,800.0,6,3.0,1.0,1.0,5000.0000,4.0,NaN
6,11,p197225,2011,75.0,3000.0,6,4.0,8.0,1.0,10500.0000,13.0,NaN
7,11,p60345,1995,54.0,14200.0,5,0.0,2.0,2.0,11280.0000,5.0,NaN
8,11,p124661,2002,48.0,2800.0,7,0.0,22.0,4.0,100.0000,0.0,NaN
9,11,p107376,2004,51.0,50.0,6,0.0,2.0,2.0,100.0000,0.0,NaN


In [17]:
EID_HY_ETYPE = entbase[['EID', 'HY', 'ETYPE']]

In [18]:
entbase.TZINUM.unique()

array([ nan,   3.,   1.,   2.,   8.,   6.,   4.,   5.,  11.,   7.,  12.,
        15.,  10.,  16.,  14.,   9.,  18.,  13.,  27.,  24.,  25.,  19.])

In [19]:
entbase['EID_NUMBER'] = entbase.EID.apply(lambda row: row[1:])
entbase['EID_NUMBER'] = entbase.EID.apply(lambda row: row[1:])

In [20]:
entbase.ZCZB.fillna(0, inplace = True)
entbase.MPNUM.fillna(0, inplace = True)
entbase.INUM.fillna(0, inplace = True)
entbase.ENUM.fillna(0, inplace = True)
entbase.FINZB.fillna(0, inplace = True)
entbase.FSTINUM.fillna(0, inplace = True)
entbase.TZINUM.fillna(0, inplace = True)


new_entbase = entbase.copy()
new_entbase['RGYEAR_gap'] = 2016 - new_entbase.RGYEAR
new_entbase['ZCZB_mul_YEAR'] = new_entbase['RGYEAR_gap'] * 100 / 65 * new_entbase['ZCZB']

In [22]:
new_entbase['2014_Policy'] = new_entbase['RGYEAR'].apply(lambda row: 1 if row < 2014 else 2)

In [571]:
HY_null_index = list(new_entbase[new_entbase.HY.isnull()].index)

In [572]:
for index in HY_null_index:
    row = new_entbase.loc[index]
    etype = row.ETYPE
    etype_record = new_entbase[new_entbase.ETYPE == etype]
    fillHY = (etype_record.HY.value_counts().reset_index())['index'][0]
    row.HY = fillHY
    new_entbase.loc[index] = row

/opt/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [573]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
tmp = new_entbase.copy()
tmp.drop(['EID','EID_NUMBER', 'TZINUM'], axis = 1, inplace = True)
poly_feature = poly.fit_transform(tmp)
colname = [('Poly_f' + str(i)) for i in range(poly_feature.shape[1])]

In [574]:
ploy_df = pd.DataFrame(poly_feature, columns = colname)
new_entbase = pd.concat([new_entbase, ploy_df], axis = 1)

In [575]:
new_entbase['n_ZCZB_DEL_FINZB'] = new_entbase['ZCZB_mul_YEAR'] - new_entbase['FINZB']
new_entbase['n_ZCZB_ADD_FINZB'] = new_entbase['ZCZB_mul_YEAR'] + new_entbase['FINZB']
new_entbase['MP2-FSTI2'] = new_entbase['MPNUM']**2 - new_entbase['FSTINUM']**2

#1.10
new_entbase['MP2+FSTI2'] = new_entbase['MPNUM']**2 + new_entbase['FSTINUM']**2
new_entbase['I2+FSTI2'] = new_entbase['INUM']**2 + new_entbase['FSTINUM']**2
new_entbase['E2+FSTI2'] = new_entbase['ENUM']**2 + new_entbase['FSTINUM']**2
new_entbase['I2+MP2'] = new_entbase['INUM']**2 + new_entbase['MPNUM']**2
new_entbase['E2+MP2'] = new_entbase['ENUM']**2 + new_entbase['MPNUM']**2

new_entbase['RGYEAR_mul_FINZB'] = new_entbase['RGYEAR_gap'] * new_entbase['FINZB']
new_entbase['RGYEAR_mul_I'] = new_entbase['RGYEAR_gap'] * new_entbase['INUM']
new_entbase['RGYEAR_mul_MP'] = new_entbase['RGYEAR_gap'] * new_entbase['MPNUM']
new_entbase['RGYEAR_mul_FSTI'] = new_entbase['RGYEAR_gap'] * new_entbase['FSTINUM']
new_entbase['RGYEAR_mul_E'] = new_entbase['RGYEAR_gap'] * new_entbase['ENUM']

In [576]:
new_entbase.MPNUM.unique().max(),new_entbase.INUM.unique().max(),new_entbase.ENUM.unique().max(),new_entbase.FSTINUM.unique().max(),\
new_entbase.TZINUM.unique().max()


(49261.0, 1900.0, 391.0, 27480.0, 27.0)

In [577]:
new_entbase.shape

(436798, 119)

version3

In [578]:
ZCZB_mul_YEAR_HandE = get_HandEforEntbase(new_entbase, 'ZCZB_mul_YEAR', len('ZCZB_mul_YEAR'))
MPNUM_HandE = get_HandEforEntbase(ZCZB_mul_YEAR_HandE, 'MPNUM', len('MPNUM'))
INUM_HandE = get_HandEforEntbase(MPNUM_HandE, 'INUM', len('INUM'))
FSTINUM_HandE = get_HandEforEntbase(INUM_HandE, 'FSTINUM', len('FSTINUM'))
RGYEAR_HandE = get_HandEforEntbase(FSTINUM_HandE, 'RGYEAR', len('RGYEAR'))


ZCZB_mul_YEAR_HandEandP  = get_HandEandPforEntbase(RGYEAR_HandE, 'ZCZB_mul_YEAR', len('ZCZB_mul_YEAR'))
MPNUM_HandEandP  = get_HandEandPforEntbase(ZCZB_mul_YEAR_HandEandP, 'MPNUM', len('MPNUM'))
INUM_HandEandP  = get_HandEandPforEntbase(MPNUM_HandEandP, 'INUM', len('INUM'))
FSTINUM_HandEandP  = get_HandEandPforEntbase(INUM_HandEandP, 'FSTINUM', len('FSTINUM'))
RGYEAR_HandEandP = get_HandEandPforEntbase(FSTINUM_HandEandP, 'RGYEAR', len('RGYEAR'))

#new_entbase2 = FSTINUM_HandE.copy()
new_entbase2 = RGYEAR_HandEandP.copy()


In [579]:
new_entbase2.shape

(436798, 259)

ver3.6

In [580]:
entbaseDAE = new_entbase2.copy()

In [581]:
entbaseDAE

,PROV,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,RGYEAR_PROVandETYPE_median,RGYEAR_PROVandETYPE_max,RGYEAR_PROVandETYPE_mean,RGYEAR_PROVandHYandETYPE_median,RGYEAR_PROVandHYandETYPE_max,RGYEAR_PROVandHYandETYPE_mean,RGYEAR_PROV_gap,RGYEAR_PROVandHY_gap,RGYEAR_PROVandETYPE_gap,RGYEAR_PROVandHYandETYPE_gap
0,11,p13836,1980,51.0,480.0,13,25.0,1.0,1.0,98041.0000,...,1992,2015,1991.951526,1990.0,2013,1990.233846,-30.517320,-31.074920,-11.951526,-10.233846
1,11,p49137,1993,74.0,30.0,13,0.0,1.0,1.0,570.0000,...,1992,2015,1991.951526,1994.0,2015,1995.777778,-17.517320,-17.339322,1.048474,-2.777778
2,11,p108756,2015,69.0,1000.0,6,2.0,1.0,1.0,1000.0000,...,2013,2015,2010.969033,2014.0,2015,2014.189591,4.482680,1.092655,4.030967,0.810409
3,11,p7320,2005,74.0,240.0,5,0.0,2.0,2.0,1500.0000,...,2007,2015,2005.810045,2010.0,2015,2008.422222,-5.517320,-5.339322,-0.810045,-3.422222
4,11,p194070,2014,51.0,100000.0,3,3.0,2.0,2.0,1000.0000,...,2010,2015,2007.657556,2006.0,2015,2006.250000,3.482680,2.925080,6.342444,7.750000
5,11,p120116,2009,51.0,800.0,6,3.0,1.0,1.0,5000.0000,...,2013,2015,2010.969033,2013.0,2015,2011.358401,-1.517320,-2.074920,-1.969033,-2.358401
6,11,p197225,2011,75.0,3000.0,6,4.0,8.0,1.0,10500.0000,...,2013,2015,2010.969033,2010.0,2015,2009.810354,0.482680,0.031959,0.030967,1.189646
7,11,p60345,1995,54.0,14200.0,5,0.0,2.0,2.0,11280.0000,...,2007,2015,2005.810045,1996.0,2015,1995.432692,-15.517320,-13.901329,-10.810045,-0.432692
8,11,p124661,2002,48.0,2800.0,7,0.0,22.0,4.0,100.0000,...,2013,2015,2011.236393,2013.0,2015,2011.465116,-8.517320,-8.059871,-9.236393,-9.465116
9,11,p107376,2004,51.0,50.0,6,0.0,2.0,2.0,100.0000,...,2013,2015,2010.969033,2013.0,2015,2011.358401,-6.517320,-7.074920,-6.969033,-7.358401


In [582]:
train1 = pd.merge(new_train, entbaseDAE, on = 'EID', how = 'left')
test1 = pd.merge(test, entbaseDAE, on = 'EID', how = 'left')

# Alter Feature

In [583]:
alter = pd.read_csv(data_path + '2alter.csv')

In [584]:
alter

,EID,ALTERNO,ALTDATE,ALTBE,ALTAF
0,p2,01,2015-01,NaN,NaN
1,p2,04,2015-01,NaN,NaN
2,p2,10,2015-01,NaN,NaN
3,p2,12,2014-08,NaN,NaN
4,p2,12,2015-01,NaN,NaN
5,p11,13,2014-07,NaN,NaN
6,p13,03,2015-05,NaN,NaN
7,p17,13,2014-04,NaN,NaN
8,p17,12,2014-04,NaN,NaN
9,p23,12,2015-08,NaN,NaN


In [585]:
alter.ALTERNO.unique()

array(['01', '04', '10', '12', '13', '03', '05', '02', '99', '27', 'A_015',
       '14'], dtype=object)

In [586]:
alter.ALTDATE.unique()

array(['2015-01', '2014-08', '2014-07', '2015-05', '2014-04', '2015-08',
       '2015-03', '2014-09', '2014-12', '2015-04', '2015-07', '2014-03',
       '2014-10', '2014-05', '2014-11', '2014-06', '2015-02', '2015-06',
       '2014-02', '2013-11', '2013-09', '2013-12', '2014-01', '2013-10'], dtype=object)

In [587]:
import re

In [588]:
from sklearn import preprocessing  

cdf = alter.copy()
label_enc = preprocessing.LabelEncoder()
label_enc.fit(alter.ALTERNO.values)
cdf.ALTERNO = label_enc.transform(alter.ALTERNO.values)
alterno_05, alterno_27 = label_enc.transform(['05', '27']) 
cdf['ALTDATE_LE'] = label_enc.fit_transform(alter.ALTDATE.values)
cdf['ALTDATE'] = cdf['ALTDATE'].apply(translateYear)

new_df = cdf.groupby(['EID'], as_index = 0)['ALTERNO'].count()
new_df.columns = ['EID', 'ALT_NUM']
ALTNO_VC = cdf.groupby(['EID'])['ALTERNO'].value_counts().unstack().fillna(0).add_prefix('ALTERNO_').reset_index()

In [589]:
cdf

,EID,ALTERNO,ALTDATE,ALTBE,ALTAF,ALTDATE_LE
0,p2,0,1,NaN,NaN,16
1,p2,3,1,NaN,NaN,16
2,p2,5,1,NaN,NaN,16
3,p2,6,-4,NaN,NaN,11
4,p2,6,1,NaN,NaN,16
5,p11,7,-5,NaN,NaN,10
6,p13,2,5,NaN,NaN,20
7,p17,7,-8,NaN,NaN,7
8,p17,6,-8,NaN,NaN,7
9,p23,6,8,NaN,NaN,23


In [590]:
# 12.4 alterno label feature
tmp = cdf.copy()

tmp_train = pd.merge(train[['TARGET', 'EID']], cdf[['ALTERNO','EID']], on = 'EID', how = 'left')
tmp_train.fillna(-1, inplace = True)

count_alter_label = tmp_train.groupby(['TARGET','ALTERNO']).count().reset_index()
alterno_label_0 = count_alter_label[count_alter_label.TARGET == 0]
alterno_label_1 = count_alter_label[count_alter_label.TARGET == 1]
alterno_label_0.rename(columns = {'EID': 'ALTERNO_label_0_count'},inplace = True)
alterno_label_1.rename(columns = {'EID': 'ALTERNO_label_1_count'},inplace = True)


alterno_label_count = pd.merge(alterno_label_0, alterno_label_1, on = 'ALTERNO', how = 'left')
alterno_label_count.drop(['TARGET_x', 'TARGET_y'], axis = 1, inplace = True)

alterno_label_count

def get_AlTERNO_LogOdds(row):
    label_0 = row.ALTERNO_label_0_count
    label_1 = row.ALTERNO_label_1_count
    all_count = label_0 + label_1
    c = 0.01
    logOdds = math.log(label_0 + c * label_0 / all_count) - math.log(label_1 + c * label_1 / all_count)
    return logOdds

alterno_label_count['ALTERNO_label_LogOdds'] = alterno_label_count.apply(get_AlTERNO_LogOdds, axis = 1)
alterno_label_count['ALTERNO_label_1_rate'] = alterno_label_count['ALTERNO_label_1_count'] * 1.0 / (alterno_label_count['ALTERNO_label_1_count']+ alterno_label_count['ALTERNO_label_0_count'])


/opt/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [591]:
alterno_label_count

,ALTERNO,ALTERNO_label_0_count,ALTERNO_label_1_count,ALTERNO_label_LogOdds,ALTERNO_label_1_rate
0,-1.0,128590,21752,1.776923,0.144683
1,0.0,7524,1744,1.461917,0.188174
2,1.0,15666,2289,1.923378,0.127485
3,2.0,18125,4057,1.496848,0.182896
4,3.0,7217,1541,1.544008,0.175953
5,4.0,16158,2409,1.903204,0.129746
6,5.0,23668,4308,1.703650,0.153989
7,6.0,38617,5535,1.942601,0.125362
8,7.0,3476,402,2.157185,0.103662
9,8.0,394,41,2.262779,0.094253


ALTNO 
05, 27

In [592]:
cdf['ALTGAP'] = cdf.ALTAF - cdf.ALTBE
cdf_05 = cdf[cdf.ALTERNO == alterno_05]
cdf_27 = cdf[cdf.ALTERNO == alterno_27]

In [593]:
cdf_05

,EID,ALTERNO,ALTDATE,ALTBE,ALTAF,ALTDATE_LE,ALTGAP
21,p47,4,7,100.00,1000.0000,22,900.0000
25,p50,4,-3,50.00,200.0000,12,150.0000
27,p53,4,1,10.00,1200.0000,16,1190.0000
47,p77,4,7,10.00,300.0000,22,290.0000
51,p89,4,3,100.00,500.0000,18,400.0000
54,p92,4,-1,50.00,500.0000,14,450.0000
67,p104,4,-2,10.00,50.0000,13,40.0000
69,p108,4,-3,50.00,800.0000,12,750.0000
76,p131,4,1,50.00,300.0000,16,250.0000
81,p132,4,-6,50.00,100.0000,9,50.0000


In [594]:
#version 2.2     ver4.1  add sum
cdf_05_be = cdf_05.groupby(['EID'])['ALTBE'].agg([sum, max, min, np.mean])
cdf_05_af = cdf_05.groupby(['EID'])['ALTAF'].agg([sum, max, min, np.mean])
cdf_27_be = cdf_27.groupby(['EID'])['ALTBE'].agg([sum, max, min, np.mean])
cdf_27_af = cdf_27.groupby(['EID'])['ALTAF'].agg([sum, max, min, np.mean])

cdf_05_count = cdf_05.groupby(['EID'])['ALTERNO'].count()
cdf_27_count = cdf_27.groupby(['EID'])['ALTERNO'].count()

In [595]:
#ver4.2  add range
cdf_05_be['range'] = cdf_05_be['max'] - cdf_05_be['min']
cdf_05_af['range'] = cdf_05_af['max'] - cdf_05_af['min']
cdf_27_be['range'] = cdf_27_be['max'] - cdf_27_be['min']
cdf_27_af['range'] = cdf_05_be['max'] - cdf_27_af['min']

In [596]:
cdf_05_be = cdf_05_be.add_prefix('ALTNO_05_BE_').reset_index()
cdf_05_af = cdf_05_af.add_prefix('ALTNO_05_AF_').reset_index()
cdf_27_be = cdf_27_be.add_prefix('ALTNO_27_BE_').reset_index()
cdf_27_af = cdf_27_af.add_prefix('ALTNO_27_AF_').reset_index()
cdf_05_count = cdf_05_count.reset_index().rename(columns = {'ALTERNO': 'ALTNO_05_Count'})
cdf_27_count = cdf_27_count.reset_index().rename(columns = {'ALTERNO': 'ALTNO_27_Count'})

In [597]:
new_df1 = pd.merge(new_df, ALTNO_VC, on = 'EID', how = 'left')
new_df2 = pd.merge(new_df1, cdf_05_be, on = 'EID', how = 'left')
new_df3 = pd.merge(new_df2, cdf_05_af, on = 'EID', how = 'left')
new_df4 = pd.merge(new_df3, cdf_27_be, on = 'EID', how = 'left')
new_df5 = pd.merge(new_df4, cdf_27_af, on = 'EID', how = 'left')
new_df6 = pd.merge(new_df5, cdf_05_count, on = 'EID', how = 'left')
new_df7 = pd.merge(new_df6, cdf_27_count, on = 'EID', how = 'left')

In [598]:
new_df7

,EID,ALT_NUM,ALTERNO_0,ALTERNO_1,ALTERNO_2,ALTERNO_3,ALTERNO_4,ALTERNO_5,ALTERNO_6,ALTERNO_7,...,ALTNO_27_BE_min,ALTNO_27_BE_mean,ALTNO_27_BE_range,ALTNO_27_AF_sum,ALTNO_27_AF_max,ALTNO_27_AF_min,ALTNO_27_AF_mean,ALTNO_27_AF_range,ALTNO_05_Count,ALTNO_27_Count
0,p10000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100004,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,p100007,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100010,4,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,p100016,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
5,p100018,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,p100022,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
7,p100031,3,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
8,p100034,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,p100043,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


version 2   (效果好)

In [599]:
# v1
"""
cdf_05_gap = cdf_05.groupby(['EID'])['ALTGAP'].agg({'max': np.max, 'min': np.min, 'mean': np.mean, 'median': np.median})
cdf_27_gap = cdf_27.groupby(['EID'])['ALTGAP'].agg({'max': np.max, 'min': np.min, 'mean': np.mean, 'median': np.median})"""
# version 2.2   v4  add sum
cdf_05_gap = cdf_05.groupby(['EID'])['ALTGAP'].agg([sum, max, min, np.mean])
cdf_27_gap = cdf_27.groupby(['EID'])['ALTGAP'].agg([sum, max, min, np.mean])

#v4.2  add range
cdf_05_gap['range'] = cdf_05_gap['max'] - cdf_05_gap['min']
cdf_27_gap['range'] = cdf_27_gap['max'] - cdf_27_gap['min']

cdf_05_gap = cdf_05_gap.add_prefix('ALTNO_05_GAP_').reset_index()
cdf_27_gap = cdf_27_gap.add_prefix('ALTNO_27_GAP_').reset_index()

new_df8 = pd.merge(new_df7, cdf_05_gap, on = 'EID', how = 'left')
new_df9 = pd.merge(new_df8, cdf_27_gap, on = 'EID', how = 'left')

# *** 
cdf2015 = cdf[cdf.ALTDATE >= 61]  # 61: 2015-01
cdf2015_alter_sum = cdf2015.groupby(['EID']).size().reset_index().rename(columns = {0: 'alter2015_sum'})

df_2015 = pd.merge(new_df, cdf2015_alter_sum, on='EID', how = 'left').fillna(0)
df_2015['alter2015_rate'] = df_2015['alter2015_sum'] / df_2015['ALT_NUM']
df_2015 = df_2015.drop(['ALT_NUM'], axis = 1)

new_df10 = pd.merge(new_df9, df_2015, on = 'EID', how = 'left')

v4.1

In [600]:
alter

,EID,ALTERNO,ALTDATE,ALTBE,ALTAF
0,p2,01,2015-01,NaN,NaN
1,p2,04,2015-01,NaN,NaN
2,p2,10,2015-01,NaN,NaN
3,p2,12,2014-08,NaN,NaN
4,p2,12,2015-01,NaN,NaN
5,p11,13,2014-07,NaN,NaN
6,p13,03,2015-05,NaN,NaN
7,p17,13,2014-04,NaN,NaN
8,p17,12,2014-04,NaN,NaN
9,p23,12,2015-08,NaN,NaN


In [601]:
# 计算最近两次更改的时间差  数值越小代表越频繁
eid_altdate_df = pd.read_csv(data_path + 'eid_alterdate_gap.csv')
new_df11 = pd.merge(new_df10, eid_altdate_df, on = 'EID', how = 'left')

v4.2

In [602]:
alter_date_sta = cdf.groupby(['EID'])['ALTDATE'].agg([max, min]).add_prefix('alter_ALTDATE_').reset_index()
alter_date_sta['alter_ALTDATE_range'] = alter_date_sta['alter_ALTDATE_max'] - alter_date_sta['alter_ALTDATE_min']
new_df12 = pd.merge(new_df11, alter_date_sta, on = 'EID', how = 'left')

In [603]:
# 12.4 alterno label feature
new_cdf = pd.merge(cdf, alterno_label_count, on = 'ALTERNO', how = 'left')
new_cdf.drop(['ALTERNO_label_0_count', 'ALTERNO_label_1_count'], axis = 1, inplace = True)

ALTERNO_avg_LogOdds = new_cdf.groupby(['EID'])['ALTERNO_label_LogOdds'].mean().reset_index()
ALTERNO_avg_label_1_rate = new_cdf.groupby(['EID'])['ALTERNO_label_1_rate'].mean().reset_index()

new_df13 = pd.merge(new_df12, ALTERNO_avg_LogOdds, on = 'EID', how ='left')
new_df14 = pd.merge(new_df13, ALTERNO_avg_label_1_rate, on = 'EID', how ='left')

In [604]:
alterDAE = new_df14.copy()

In [605]:
train2 = pd.merge(train1, alterDAE, on = 'EID', how = 'left')
test2 = pd.merge(test1, alterDAE, on = 'EID', how = 'left' )
train2.fillna(0, inplace = True)
test2.fillna(0, inplace = True)

# branch feature

In [606]:
branch = pd.read_csv(data_path + '3branch.csv')

In [607]:
branch

,EID,TYPECODE,IFHOME,B_REYEAR,B_ENDYEAR
0,p2,pbr32949,0,2008,NaN
1,p2,pbr28271,0,2004,2004.0
2,p2,pbr14086,0,2005,2008.0
3,p2,pbr9435,0,2010,2015.0
4,p56,pbr1,1,2011,NaN
5,p56,pbr18805,1,2012,NaN
6,p166,pbr18806,1,2015,NaN
7,p235,pbr32950,1,2008,2009.0
8,p572,pbr14087,1,2013,NaN
9,p572,pbr9436,1,2014,2015.0


In [608]:
# 关停的年份
branch.B_REYEAR.unique()

array([2008, 2004, 2005, 2010, 2011, 2012, 2015, 2013, 2014, 1999, 2003,
       2001, 2000, 2002, 2009, 2006, 1996, 1984, 1991, 2007, 1997, 1989,
       1998, 1986, 1985, 1992, 1993, 1995, 1994, 1988, 1990, 1987, 1982,
       1983, 1981, 1967, 1980, 1976, 1978, 1962, 1959, 1977, 1899, 1979,
       1900, 1974, 1969, 1973, 1975, 1950, 1955, 1972, 1956, 1965, 1964,
       1952, 1960, 1971, 1970, 1954, 1963, 1949, 1958])

In [609]:
# 关停的年份
branch.B_ENDYEAR.unique()

array([   nan,  2004.,  2008.,  2015.,  2009.,  2014.,  2013.,  2012.,
        2006.,  2007.,  2002.,  2005.,  2010.,  1997.,  2003.,  2011.,
        2000.,  1994.,  2001.,  1998.,  1996.,  1995.,  1999.,  1992.,
        1990.,  1987.,  1989.,  1993.,  1991.,  1986.,  1950.,  1988.,
        1985.,  1899.,  1949.,  1900.])

In [610]:
# 439公司有315家分公司记录
branch[branch.EID == 439]

,EID,TYPECODE,IFHOME,B_REYEAR,B_ENDYEAR


In [611]:
branch['TYPECODE'] = branch['TYPECODE'].apply(lambda row: row[2:])

In [612]:
len(branch.EID.unique())

27736

In [613]:
new_branch = branch.groupby(['EID'])['IFHOME'].count()
new_branch = new_branch.reset_index().rename(columns = {'IFHOME': "Branch_Count"})

bstart_count = branch.groupby(['EID'])['B_REYEAR'].count().reset_index().rename(columns = {'B_REYEAR': "Bstart_count"})
bend_count = branch[branch.B_ENDYEAR.isnull() == False].groupby(['EID'])['B_ENDYEAR'].count().reset_index().rename(columns = {'B_ENDYEAR': 'Bend_count'})
bhome_count = branch.groupby(['EID', 'IFHOME'])['TYPECODE'].count().unstack().add_prefix('BRANCH_IFHOME_').add_suffix('_count').reset_index()

new_branch1 = pd.merge(new_branch, bstart_count, on = 'EID', how = 'left')
new_branch2 = pd.merge(new_branch1, bend_count, on = 'EID', how = 'left')
new_branch3 = pd.merge(new_branch2, bhome_count, on = 'EID', how = 'left')

new_branch3['BRANCH_IFHOME_0_rate'] = new_branch3['BRANCH_IFHOME_0_count'] / new_branch3['Branch_Count']
new_branch3['BRANCH_IFHOME_1_rate'] = new_branch3['BRANCH_IFHOME_1_count'] / new_branch3['Branch_Count']
new_branch3['Bend_rate'] = new_branch3['Bend_count'] / new_branch3['Branch_Count']


In [614]:
new_branch

,EID,Branch_Count
0,p100035,1
1,p100037,1
2,p100118,2
3,p100164,19
4,p100187,7
5,p100205,1
6,p100206,2
7,p100230,1
8,p100266,1
9,p100304,1


version2 (有效)

In [615]:
#**
branch_start_2013 = branch[branch.B_REYEAR >= 2013].groupby(['EID']).size().reset_index().rename(columns = {0 : 'branch_start_from2013_count'})
branch_end_2013 = branch[branch.B_ENDYEAR >= 2013].groupby(['EID']).size().reset_index().rename(columns = {0 : 'branch_end_from2013_count'})

new_branch4 = pd.merge(new_branch3, branch_start_2013, on = 'EID', how = 'left')
new_branch5 = pd.merge(new_branch4, branch_end_2013, on = 'EID', how = 'left')

new_branch5.fillna(0, inplace = True)

version3

In [616]:
last_branch = branch[branch.B_ENDYEAR.isnull()]['B_REYEAR'].groupby(branch.EID).max().reset_index()
new_branch6 = pd.merge(new_branch5, last_branch, on = 'EID', how = 'left')

new_branch6.fillna(-1, inplace = True)

ver4.2

In [617]:
B_REYEAR_sta = branch['B_REYEAR'].groupby(branch.EID).agg([max, min])
B_REYEAR_sta['range'] = B_REYEAR_sta['max'] - B_REYEAR_sta['min']
B_REYEAR_sta = B_REYEAR_sta.add_prefix('branch_B_REYEAR_').reset_index()

B_ENDYEAR_sta = branch['B_ENDYEAR'].groupby(branch.EID).agg([max, min])
B_ENDYEAR_sta['range'] = B_ENDYEAR_sta['max'] - B_ENDYEAR_sta['min']
B_ENDYEAR_sta = B_ENDYEAR_sta.add_prefix('branch_B_ENDYEAR_').reset_index()

new_branch7 = pd.merge(new_branch6, B_REYEAR_sta, on = 'EID', how = 'left')
new_branch8 = pd.merge(new_branch7, B_ENDYEAR_sta, on = 'EID', how = 'left')

In [618]:
new_branch8

,EID,Branch_Count,Bstart_count,Bend_count,BRANCH_IFHOME_0_count,BRANCH_IFHOME_1_count,BRANCH_IFHOME_0_rate,BRANCH_IFHOME_1_rate,Bend_rate,branch_start_from2013_count,branch_end_from2013_count,B_REYEAR,branch_B_REYEAR_max,branch_B_REYEAR_min,branch_B_REYEAR_range,branch_B_ENDYEAR_max,branch_B_ENDYEAR_min,branch_B_ENDYEAR_range
0,p100035,1,1,0.0,0.0,1.0,0.000000,1.000000,0.000000,1.0,0.0,2014.0,2014,2014,0,NaN,NaN,NaN
1,p100037,1,1,0.0,0.0,1.0,0.000000,1.000000,0.000000,1.0,0.0,2013.0,2013,2013,0,NaN,NaN,NaN
2,p100118,2,2,1.0,0.0,2.0,0.000000,1.000000,0.500000,0.0,0.0,2008.0,2008,2008,0,2011.0,2011.0,0.0
3,p100164,19,19,1.0,0.0,19.0,0.000000,1.000000,0.052632,11.0,1.0,2015.0,2015,2010,5,2015.0,2015.0,0.0
4,p100187,7,7,0.0,0.0,7.0,0.000000,1.000000,0.000000,7.0,0.0,2014.0,2014,2013,1,NaN,NaN,NaN
5,p100205,1,1,0.0,0.0,1.0,0.000000,1.000000,0.000000,0.0,0.0,2008.0,2008,2008,0,NaN,NaN,NaN
6,p100206,2,2,0.0,0.0,2.0,0.000000,1.000000,0.000000,1.0,0.0,2015.0,2015,2012,3,NaN,NaN,NaN
7,p100230,1,1,1.0,0.0,1.0,0.000000,1.000000,1.000000,0.0,0.0,-1.0,2006,2006,0,2012.0,2012.0,0.0
8,p100266,1,1,1.0,0.0,1.0,0.000000,1.000000,1.000000,0.0,0.0,-1.0,2008,2008,0,2011.0,2011.0,0.0
9,p100304,1,1,1.0,0.0,1.0,0.000000,1.000000,1.000000,0.0,0.0,-1.0,2009,2009,0,2009.0,2009.0,0.0


In [619]:
branchDAE = new_branch8.copy()

In [620]:
train3 = pd.merge(train2, branchDAE, on = 'EID', how = 'left')
test3 = pd.merge(test2, branchDAE, on = 'EID', how = 'left')

In [621]:
branchDAE

,EID,Branch_Count,Bstart_count,Bend_count,BRANCH_IFHOME_0_count,BRANCH_IFHOME_1_count,BRANCH_IFHOME_0_rate,BRANCH_IFHOME_1_rate,Bend_rate,branch_start_from2013_count,branch_end_from2013_count,B_REYEAR,branch_B_REYEAR_max,branch_B_REYEAR_min,branch_B_REYEAR_range,branch_B_ENDYEAR_max,branch_B_ENDYEAR_min,branch_B_ENDYEAR_range
0,p100035,1,1,0.0,0.0,1.0,0.000000,1.000000,0.000000,1.0,0.0,2014.0,2014,2014,0,NaN,NaN,NaN
1,p100037,1,1,0.0,0.0,1.0,0.000000,1.000000,0.000000,1.0,0.0,2013.0,2013,2013,0,NaN,NaN,NaN
2,p100118,2,2,1.0,0.0,2.0,0.000000,1.000000,0.500000,0.0,0.0,2008.0,2008,2008,0,2011.0,2011.0,0.0
3,p100164,19,19,1.0,0.0,19.0,0.000000,1.000000,0.052632,11.0,1.0,2015.0,2015,2010,5,2015.0,2015.0,0.0
4,p100187,7,7,0.0,0.0,7.0,0.000000,1.000000,0.000000,7.0,0.0,2014.0,2014,2013,1,NaN,NaN,NaN
5,p100205,1,1,0.0,0.0,1.0,0.000000,1.000000,0.000000,0.0,0.0,2008.0,2008,2008,0,NaN,NaN,NaN
6,p100206,2,2,0.0,0.0,2.0,0.000000,1.000000,0.000000,1.0,0.0,2015.0,2015,2012,3,NaN,NaN,NaN
7,p100230,1,1,1.0,0.0,1.0,0.000000,1.000000,1.000000,0.0,0.0,-1.0,2006,2006,0,2012.0,2012.0,0.0
8,p100266,1,1,1.0,0.0,1.0,0.000000,1.000000,1.000000,0.0,0.0,-1.0,2008,2008,0,2011.0,2011.0,0.0
9,p100304,1,1,1.0,0.0,1.0,0.000000,1.000000,1.000000,0.0,0.0,-1.0,2009,2009,0,2009.0,2009.0,0.0


# Invest Feature

In [622]:
invest = pd.read_csv(data_path + '4invest.csv')

In [623]:
invest

,EID,BTEID,IFHOME,BTBL,BTYEAR,BTENDYEAR
0,p17402,s102046,0,0.343,2012,NaN
1,p89336,s140778,0,1.000,2012,NaN
2,p142507,s62517,0,1.000,2011,NaN
3,p218390,s305106,0,0.800,2015,NaN
4,p134285,s425895,0,0.011,2014,NaN
5,p101545,s437754,0,1.000,2015,NaN
6,p17402,s107413,0,0.250,2015,NaN
7,p144466,s80568,0,0.630,1999,NaN
8,p150675,s521154,0,0.200,2013,NaN
9,p60187,s343826,0,0.500,2014,NaN


In [624]:
new_invest = invest.groupby(['EID']).size().reset_index().rename(columns = {0: 'invest_count'})
invest_home = invest.groupby(['EID', 'IFHOME'])['BTEID'].count().unstack().add_prefix('INVEST_IFHOME_').add_suffix('_count').reset_index()
#be_invested_count = invest.groupby(['BTEID']).size().reset_index().rename(columns = {0: 'be_invested_count', 'BTEID': 'EID'})
#be_invested_home = invest.groupby(['BTEID', 'IFHOME'])['EID'].count().unstack().add_prefix('BE_INVESTED_IFHOME_').add_suffix('_count').reset_index().rename(columns = {'BTEID': 'EID'})
invest_btbl = invest.groupby(['EID'])['BTBL'].agg([sum, max, min, np.mean, np.median]).add_prefix('invest_btbl_').reset_index()
invest_closed = invest[invest.BTENDYEAR.isnull() == False].groupby(['EID']).size().reset_index().rename(columns = {0: 'invest_closed_count'})

new_invest1 = pd.merge(new_invest, invest_home, on = 'EID', how = 'left')
new_invest4 = pd.merge(new_invest1, invest_btbl, on = 'EID', how = 'left')
new_invest5 = pd.merge(new_invest4, invest_closed, on = 'EID', how = 'left')

new_invest5['INVEST_IFHOME_0_rate'] = new_invest5['INVEST_IFHOME_0_count'] / new_invest5['invest_count']
new_invest5['INVEST_IFHOME_1_rate'] = new_invest5['INVEST_IFHOME_1_count'] / new_invest5['invest_count']
new_invest5['invest_closed_rate'] = new_invest5['invest_closed_count'] / new_invest5['invest_count']

new_invest5.fillna(0, inplace = True)

version 2 .1

In [625]:
#*
invest_2014_not_end = invest[invest.BTENDYEAR.isnull()]
invest_after_2014 = invest_2014_not_end[invest_2014_not_end.BTYEAR > 2014]
invest_after_2014_count = invest_after_2014.groupby(['EID']).size().reset_index().rename(columns = {0: 'invest_from2014_count'})
invest_btbl_after2014 = invest_after_2014.groupby(['EID'])['BTBL'].agg([max, min, np.mean]).add_prefix('invest_btbl_after_2014_').reset_index()

#**
invest_before_2008 = invest[invest.BTYEAR < 2008]
invest_before_2008_count = invest_before_2008.groupby(['EID']).size().reset_index().rename(columns = {0: 'invest_before2008_count'})
invest_btbl_before2008 = invest_before_2008.groupby(['EID'])['BTBL'].agg([max, min, np.mean]).add_prefix('invest_btbl_before_2008_').reset_index()


new_invest6 = pd.merge(new_invest5, invest_after_2014_count, on = 'EID', how = 'left')
new_invest7 = pd.merge(new_invest6, invest_btbl_after2014, on = 'EID', how = 'left')
new_invest8 = pd.merge(new_invest7, invest_before_2008_count, on = 'EID', how = 'left')
new_invest9 = pd.merge(new_invest8, invest_btbl_before2008, on = 'EID', how = 'left')

new_invest9.fillna(0, inplace = True)


version 3.3

In [626]:
investDAE = new_invest9.copy()

In [627]:
investDAE

,EID,invest_count,INVEST_IFHOME_0_count,INVEST_IFHOME_1_count,invest_btbl_sum,invest_btbl_max,invest_btbl_min,invest_btbl_mean,invest_btbl_median,invest_closed_count,...,INVEST_IFHOME_1_rate,invest_closed_rate,invest_from2014_count,invest_btbl_after_2014_max,invest_btbl_after_2014_min,invest_btbl_after_2014_mean,invest_before2008_count,invest_btbl_before_2008_max,invest_btbl_before_2008_min,invest_btbl_before_2008_mean
0,p100164,4,0.0,4.0,4.000,1.000,1.000,1.000000,1.0000,0.0,...,1.000000,0.000000,4.0,1.000,1.000,1.000,0.0,0.000,0.000,0.000000
1,p100184,1,0.0,1.0,0.033,0.033,0.033,0.033000,0.0330,0.0,...,1.000000,0.000000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000000
2,p10019,5,2.0,3.0,4.100,1.000,0.500,0.820000,1.0000,0.0,...,0.600000,0.000000,1.0,0.600,0.600,0.600,0.0,0.000,0.000,0.000000
3,p100206,2,0.0,2.0,1.341,1.000,0.341,0.670500,0.6705,1.0,...,1.000000,0.500000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000000
4,p100217,1,0.0,1.0,0.225,0.225,0.225,0.225000,0.2250,0.0,...,1.000000,0.000000,1.0,0.225,0.225,0.225,0.0,0.000,0.000,0.000000
5,p100220,3,0.0,3.0,0.887,0.490,0.007,0.295667,0.3900,1.0,...,1.000000,0.333333,0.0,0.000,0.000,0.000,2.0,0.490,0.390,0.440000
6,p100234,2,0.0,2.0,1.000,0.500,0.500,0.500000,0.5000,0.0,...,1.000000,0.000000,0.0,0.000,0.000,0.000,2.0,0.500,0.500,0.500000
7,p100271,1,0.0,1.0,0.150,0.150,0.150,0.150000,0.1500,0.0,...,1.000000,0.000000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000000
8,p100283,3,0.0,3.0,1.537,0.517,0.510,0.512333,0.5100,0.0,...,1.000000,0.000000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000000
9,p100286,1,0.0,1.0,1.000,1.000,1.000,1.000000,1.0000,0.0,...,1.000000,0.000000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000000


In [628]:
train4 = pd.merge(train3, investDAE, on = 'EID', how = 'left')
test4 = pd.merge(test3, investDAE, on = 'EID', how = 'left')

# Right Feature

In [629]:
right = pd.read_csv(data_path + '5right.csv')

In [630]:

right.RIGHTTYPE.unique()

array([11, 12, 20, 30, 40, 50, 60])

In [631]:
right

,EID,RIGHTTYPE,TYPECODE,ASKDATE,FBDATE
0,s244862,11,pno1,2012-11,NaN
1,s398369,11,pno2,2009-11,NaN
2,s398369,12,pno2,2009-11,2011-12
3,s461088,11,pno2,2009-11,NaN
4,s461088,12,pno2,2009-11,2011-12
5,s198466,12,pno4,2010-02,2012-04
6,s198466,11,pno4,2010-02,NaN
7,s175206,20,pno5,2012-12,2013-06
8,s198466,12,pno6,2010-02,2012-04
9,s198466,11,pno6,2010-02,NaN


In [632]:
# 12.5 righttype label feature
tmp = right.copy()

tmp_train = pd.merge(train[['TARGET', 'EID']], tmp[['RIGHTTYPE','EID']], on = 'EID', how = 'left')
tmp_train.fillna(-1, inplace = True)

count_right_label = tmp_train.groupby(['TARGET','RIGHTTYPE']).count().reset_index()
righttype_label_0 = count_right_label[count_right_label.TARGET == 0]
righttype_label_1 = count_right_label[count_right_label.TARGET == 1]
righttype_label_0.rename(columns = {'EID': 'RIGHTTYPE_label_0_count'},inplace = True)
righttype_label_1.rename(columns = {'EID': 'RIGHTTYPE_label_1_count'},inplace = True)

count_right_label

righttype_label_count = pd.merge(righttype_label_0, righttype_label_1, on = 'RIGHTTYPE', how = 'left')
righttype_label_count.drop(['TARGET_x', 'TARGET_y'], axis = 1, inplace = True)


def get_RIGHTTYPE_LogOdds(row):
    label_0 = row.RIGHTTYPE_label_0_count
    label_1 = row.RIGHTTYPE_label_1_count
    all_count = label_0 + label_1
    c = 0.01
    logOdds = math.log(label_0 + c * label_0 / all_count) - math.log(label_1 + c * label_1 / all_count)
    return logOdds

righttype_label_count['RIGHTTYPE_label_LogOdds'] = righttype_label_count.apply(get_RIGHTTYPE_LogOdds, axis = 1)
righttype_label_count['RIGHTTYPE_label_1_rate'] = righttype_label_count['RIGHTTYPE_label_1_count'] * 1.0 / (righttype_label_count['RIGHTTYPE_label_1_count']+ righttype_label_count['RIGHTTYPE_label_0_count'])


/opt/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [633]:
right.FBDATE.unique()

array([nan, '2011-12', '2012-04', '2013-06', '2013-10', '2012-09',
       '2009-12', '2008-02', '2012-12', '2013-07', '2012-05', '2015-01',
       '2008-08', '2015-06', '2008-01', '2013-08', '2012-01', '2010-05',
       '2015-04', '2012-07', '2013-11', '2014-05', '2004-06', '2013-09',
       '2014-11', '2006-12', '2013-02', '2005-08', '2013-03', '2010-12',
       '2006-01', '2010-11', '2007-01', '2012-02', '2012-11', '2015-02',
       '2009-06', '2014-03', '2008-06', '2013-05', '2014-10', '2009-05',
       '2014-06', '2007-10', '2014-02', '2010-03', '2008-12', '2014-01',
       '2002-05', '2010-06', '2014-04', '2012-10', '2011-04', '2006-11',
       '2001-06', '2003-09', '2014-08', '2010-02', '2015-08', '2015-05',
       '2002-11', '2010-10', '2013-04', '2009-09', '2008-10', '2015-07',
       '2014-12', '2009-08', '2004-07', '2006-06', '2010-01', '2011-10',
       '2011-02', '2014-09', '2012-08', '2013-01', '2007-12', '2011-01',
       '2010-04', '2014-07', '2009-03', '2011-08', '2002-

In [634]:
new_right = right.groupby(['EID']).size().reset_index().rename(columns = {0: 'right_count'})
right_type_count = right.groupby(['EID', 'RIGHTTYPE']).size().unstack().add_prefix('right_type_').add_suffix('_count').reset_index()

new_right1 = pd.merge(new_right, right_type_count, on = 'EID', how = 'left')

new_right1.fillna(0, inplace = True)

In [635]:
right

,EID,RIGHTTYPE,TYPECODE,ASKDATE,FBDATE
0,s244862,11,pno1,2012-11,NaN
1,s398369,11,pno2,2009-11,NaN
2,s398369,12,pno2,2009-11,2011-12
3,s461088,11,pno2,2009-11,NaN
4,s461088,12,pno2,2009-11,2011-12
5,s198466,12,pno4,2010-02,2012-04
6,s198466,11,pno4,2010-02,NaN
7,s175206,20,pno5,2012-12,2013-06
8,s198466,12,pno6,2010-02,2012-04
9,s198466,11,pno6,2010-02,NaN


version 2.1

In [636]:
# ***
right_apply_after2013 = right[right.ASKDATE >= '2013-01']
right_apply_after2013_count = right_apply_after2013.groupby(['EID']).size().reset_index().rename(columns = {0: 'right_ask_after2013_count'})

# 筛选2013之后申请权利 且 获得权利的记录
right_get_after2013 = right_apply_after2013[right_apply_after2013.FBDATE.notnull()]
right_get_after2013_count = right_get_after2013.groupby(['EID']).size().reset_index().rename(columns = {0: 'right_get_after2013_count'})

right_after2013 = pd.merge(right_apply_after2013_count, right_get_after2013_count, on = 'EID', how = 'left')
right_after2013['right_get_after2013_rate'] = right_after2013['right_get_after2013_count'] / right_after2013['right_ask_after2013_count']

right_get_in2015_count = right[right.FBDATE >= '2015-01'].groupby(['EID']).size().reset_index().rename(columns = {0: 'right_get_in2015_count'})

new_right2 = pd.merge(new_right1, right_after2013, on = 'EID', how = 'left')
new_right3 = pd.merge(new_right2, right_get_in2015_count, on = 'EID', how = 'left')
new_right3.fillna(0, inplace = True)

v4.2

In [637]:
rightDATE = right[['EID']]
rightDATE['ASKDATE'] = right.ASKDATE.apply(translateYear)
rightDATE['FBDATE'] = right.FBDATE.apply(translateYear)

ASKDATE_sta = rightDATE['ASKDATE'].groupby(rightDATE.EID).agg([max, min])
ASKDATE_sta['range'] = ASKDATE_sta['max'] - ASKDATE_sta['min']
ASKDATE_sta = ASKDATE_sta.add_prefix('right_ASKDATE_').reset_index()

FBDATE_sta = rightDATE['FBDATE'].groupby(rightDATE.EID).agg([max, min])
FBDATE_sta['range'] = FBDATE_sta['max'] - FBDATE_sta['min']
FBDATE_sta = FBDATE_sta.add_prefix('right_FBDATE_').reset_index()

new_right4 = pd.merge(new_right3, ASKDATE_sta, on = 'EID', how = 'left')
new_right5 = pd.merge(new_right4, FBDATE_sta, on = 'EID', how = 'left')

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [638]:
# 12.5 righttype label feature
"""new_cdf = pd.merge(right, righttype_label_count, on = 'RIGHTTYPE', how = 'left')
new_cdf.drop(['RIGHTTYPE_label_0_count', 'RIGHTTYPE_label_1_count'], axis = 1, inplace = True)

RIGHTTYPE_avg_LogOdds = new_cdf.groupby(['EID'])['RIGHTTYPE_label_LogOdds'].mean().reset_index()
RIGHTTYPE_avg_label_1_rate = new_cdf.groupby(['EID'])['RIGHTTYPE_label_1_rate'].mean().reset_index()

new_right6 = pd.merge(new_right5, RIGHTTYPE_avg_LogOdds, on = 'EID', how ='left')
new_right7 = pd.merge(new_right6, RIGHTTYPE_avg_label_1_rate, on = 'EID', how ='left')"""

"new_cdf = pd.merge(right, righttype_label_count, on = 'RIGHTTYPE', how = 'left')\nnew_cdf.drop(['RIGHTTYPE_label_0_count', 'RIGHTTYPE_label_1_count'], axis = 1, inplace = True)\n\nRIGHTTYPE_avg_LogOdds = new_cdf.groupby(['EID'])['RIGHTTYPE_label_LogOdds'].mean().reset_index()\nRIGHTTYPE_avg_label_1_rate = new_cdf.groupby(['EID'])['RIGHTTYPE_label_1_rate'].mean().reset_index()\n\nnew_right6 = pd.merge(new_right5, RIGHTTYPE_avg_LogOdds, on = 'EID', how ='left')\nnew_right7 = pd.merge(new_right6, RIGHTTYPE_avg_label_1_rate, on = 'EID', how ='left')"

In [639]:
rightDAE = new_right5.copy()

In [640]:
train5 = pd.merge(train4, rightDAE , on = 'EID', how = 'left')
test5 = pd.merge(test4, rightDAE , on = 'EID', how = 'left')

# Project Feature

In [641]:
project = pd.read_csv(data_path + '6project.csv')

In [642]:
project

,EID,TYPECODE,DJDATE,IFHOME
0,s172273,3156047,2013-11,1
1,s172273,4006392,2014-01,0
2,s172273,4833051,2014-12,1
3,s138582,4510345,2014-09,0
4,s372279,4636040,2014-10,0
5,s372279,4006095,2014-01,0
6,s11198,3984451,2014-01,0
7,s27847,4350599,2014-07,0
8,s506176,3068317,2013-10,1
9,s464937,4192469,2014-05,1


In [643]:
new_project = project.groupby(['EID']).size().reset_index().rename(columns = {0: 'project_count'})
project_home = project.groupby(['EID', 'IFHOME'])['TYPECODE'].count().unstack().add_prefix('PROJECT_IFHOME_').add_suffix('_count').reset_index()
last_project = project['DJDATE'].groupby(project.EID).max().reset_index()
last_project['DJDATE'] = last_project['DJDATE'].apply(translateYear)

In [644]:
last_project.DJDATE.unique()

array([  7,   8,   6,  -1,   3,  -5,   1,   5, -13, -15, -12, -11,   2,
         4,  -2,   0,  -7,  -6, -10, -14,  -9,  -3,  -4,  -8])

In [645]:
new_project1 = pd.merge(new_project, project_home, on = 'EID', how = 'left')
new_project2 = pd.merge(new_project1, last_project, on = 'EID', how = 'left')

new_project2.fillna(0, inplace = True)

version 2

In [646]:
# *
project_in2015_count = project[project.DJDATE >= '2015-01'].groupby(['EID']).size().reset_index().rename(columns = {0: 'project_in2015_count'})

new_project3 = pd.merge(new_project2, project_in2015_count, on = 'EID', how = 'left')
new_project3.fillna(0, inplace = True)

In [647]:
projectDAE = new_project3.copy()

In [648]:
train6 = pd.merge(train5, projectDAE, on = 'EID', how = 'left')
test6 = pd.merge(test5, projectDAE, on = 'EID', how = 'left')

# Lawsuit Feature

In [649]:
lawsuit = pd.read_csv(data_path + '7lawsuit.csv')

In [650]:
lawsuit

,EID,TYPECODE,LAWDATE,LAWAMOUNT
0,p155,107760058,2015-08,0
1,p155,102314636,2015-05,0
2,p293,110135120,2015-08,11400
3,p293,105635464,2015-08,15800
4,p293,78284856,2014-01,34000
5,p293,85369661,2014-01,34000
6,p293,77996859,2014-01,34000
7,p293,72630399,2013-12,19000
8,p342,13910277,2014-07,171600
9,p342,93353097,2014-07,171600


In [651]:
lawsuit.LAWDATE.unique()

array(['2015-08', '2015-05', '2014-01', '2013-12', '2014-07', '2012-11',
       '2015-04', '2013-09', '2013-06', '2015-02', '2015-01', '2015-06',
       '2013-01', '2014-12', '2014-10', '2013-05', '2014-05', '2014-06',
       '2015-07', '2012-12', '2014-02', '2014-03', '2015-03', '2013-11',
       '2012-09', '2013-03', '2012-10', '2014-09', '2013-10', '2013-04',
       '2014-11', '2013-02', '2013-08', '2013-07', '2014-08', '2014-04'], dtype=object)

In [652]:
new_lawsuit = lawsuit.groupby(['EID']).size().reset_index().rename(columns = {0: 'lawsuit_count'})

lawsuit_sta = lawsuit.groupby(['EID']).LAWAMOUNT.agg([sum, max, min, np.mean, np.median])
lawsuit_sta['gap'] = lawsuit_sta['max'] -  lawsuit_sta['min']
lawsuit_sta = lawsuit_sta.add_prefix('lawsuit_LAWAMOUNT_').reset_index()
lawsuit_date = lawsuit['LAWDATE'].groupby(lawsuit.EID).agg([max, min]).add_prefix('lawsuit_LAWDATE_').reset_index()

lawsuit_date['lawsuit_LAWDATE_max'] = lawsuit_date['lawsuit_LAWDATE_max'].apply(translateYear)
lawsuit_date['lawsuit_LAWDATE_min'] = lawsuit_date['lawsuit_LAWDATE_min'].apply(translateYear)
lawsuit_date['lawsuit_LAWDATE_gap'] = lawsuit_date['lawsuit_LAWDATE_max'] - lawsuit_date['lawsuit_LAWDATE_min']

new_lawsuit1 = pd.merge(new_lawsuit, lawsuit_sta, on = 'EID', how = 'left')
new_lawsuit2 = pd.merge(new_lawsuit1, lawsuit_date, on = 'EID', how = 'left')


version 2.1

In [653]:
lawsuit_in2015 = lawsuit[lawsuit.LAWDATE >= '2015-01-01']
lawsuit_in2015_count = lawsuit_in2015.groupby(['EID']).size().reset_index().rename(columns = {0: 'lawsuit_in2015_count'})
lawsuit_in2015_sta = lawsuit_in2015.groupby(['EID'])['LAWAMOUNT'].agg([sum, max, min, np.mean])
lawsuit_in2015_sta['gap'] = lawsuit_in2015_sta['max'] -  lawsuit_in2015_sta['min']
lawsuit_in2015_sta = lawsuit_in2015_sta.add_prefix('lawsuit_LAWAMOUNT_2015_').reset_index()

new_lawsuit3 = pd.merge(new_lawsuit2, lawsuit_in2015_count, on = 'EID', how = 'left')
new_lawsuit4 = pd.merge(new_lawsuit3, lawsuit_in2015_sta, on = 'EID', how = 'left')
new_lawsuit4.fillna(0, inplace = True)

In [654]:
new_lawsuit4

,EID,lawsuit_count,lawsuit_LAWAMOUNT_sum,lawsuit_LAWAMOUNT_max,lawsuit_LAWAMOUNT_min,lawsuit_LAWAMOUNT_mean,lawsuit_LAWAMOUNT_median,lawsuit_LAWAMOUNT_gap,lawsuit_LAWDATE_max,lawsuit_LAWDATE_min,lawsuit_LAWDATE_gap,lawsuit_in2015_count,lawsuit_LAWAMOUNT_2015_sum,lawsuit_LAWAMOUNT_2015_max,lawsuit_LAWAMOUNT_2015_min,lawsuit_LAWAMOUNT_2015_mean,lawsuit_LAWAMOUNT_2015_gap
0,p100022,2,122000,61000,61000,6.100000e+04,61000.0,0,-4,-4,0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
1,p100048,2,22000,11000,11000,1.100000e+04,11000.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
2,p100074,3,8646200,5509600,17100,2.882067e+06,3119500.0,5492500,6,3,3,3.0,8646200.0,5509600.0,17100.0,2.882067e+06,5492500.0
3,p100084,2,1488400,744200,744200,7.442000e+05,744200.0,0,-11,-11,0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
4,p100107,2,2,1,1,1.000000e+00,1.0,0,-16,-16,0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
5,p100129,1,118900,118900,118900,1.189000e+05,118900.0,0,-27,-27,0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
6,p100165,1,51100,51100,51100,5.110000e+04,51100.0,0,3,3,0,1.0,51100.0,51100.0,51100.0,5.110000e+04,0.0
7,p100186,1,26700,26700,26700,2.670000e+04,26700.0,0,7,7,0,1.0,26700.0,26700.0,26700.0,2.670000e+04,0.0
8,p100253,11,3002800,3000000,0,2.729818e+05,0.0,3000000,7,-25,32,9.0,0.0,0.0,0.0,0.000000e+00,0.0
9,p100270,1,5361700,5361700,5361700,5.361700e+06,5361700.0,0,3,3,0,1.0,5361700.0,5361700.0,5361700.0,5.361700e+06,0.0


In [655]:
lawsuitDAE = new_lawsuit4.copy()

In [656]:
train7 = pd.merge(train6, lawsuitDAE, on = 'EID', how = 'left')
test7 = pd.merge(test6, lawsuitDAE, on = 'EID', how = 'left')

# Breakfaith Feautre

In [657]:
breakfaith = pd.read_csv(data_path + '8breakfaith.csv')

In [658]:
breakfaith

,EID,TYPECODE,FBDATE,SXENDDATE
0,p293,432341,2014年08月,NaN
1,p342,82982060,2014年08月,NaN
2,p342,312751,2014年08月,NaN
3,p816,98741001,2015年03月,NaN
4,p816,1349372,2015年03月,NaN
5,p825,98949341,2015年05月,NaN
6,p825,1627918,2015年05月,NaN
7,p845,724299,2014年11月,2015-07
8,p1096,60556535,2014年10月,NaN
9,p1096,64318506,2014年10月,NaN


In [659]:
new_breakfaith = breakfaith.groupby(breakfaith.EID).size().reset_index().rename(columns = {0: 'breakfaith_count'})
not_end_breakfaith = breakfaith[breakfaith.SXENDDATE.isnull()].groupby(breakfaith.EID).size().reset_index().rename(columns = {0: 'not_end_breakfaith_count'})
breakfaith_date = breakfaith.groupby(breakfaith.EID)['FBDATE'].agg([max, min]).add_prefix('breakfaith_date_').reset_index()
breakfaith_date['breakfaith_date_max'] = breakfaith_date['breakfaith_date_max'].apply(translateYearFB)
breakfaith_date['breakfaith_date_min'] = breakfaith_date['breakfaith_date_min'].apply(translateYearFB)
breakfaith_date['breakfaith_date_gap'] = breakfaith_date['breakfaith_date_max'] - breakfaith_date['breakfaith_date_min']


new_breakfaith1 = pd.merge(new_breakfaith, not_end_breakfaith, on = 'EID', how = 'left')
new_breakfaith2 = pd.merge(new_breakfaith1, breakfaith_date, on = 'EID', how = 'left')
new_breakfaith2['not_end_breakfaith_rate'] = new_breakfaith2['not_end_breakfaith_count'] / new_breakfaith2['breakfaith_count']

# rate=0 代表所有失信记录全部结束 1 代表全部没结束
new_breakfaith2.fillna(0, inplace = True)



In [660]:
breakfaithDAE = new_breakfaith2.copy()

In [661]:
train8 = pd.merge(train7, breakfaithDAE, on = 'EID', how = 'left')
test8 = pd.merge(test7, breakfaithDAE, on = 'EID', how = 'left')

In [662]:
test7

,EID,PROV,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,lawsuit_LAWAMOUNT_gap,lawsuit_LAWDATE_max,lawsuit_LAWDATE_min,lawsuit_LAWDATE_gap,lawsuit_in2015_count,lawsuit_LAWAMOUNT_2015_sum,lawsuit_LAWAMOUNT_2015_max,lawsuit_LAWAMOUNT_2015_min,lawsuit_LAWAMOUNT_2015_mean,lawsuit_LAWAMOUNT_2015_gap
0,p10000,11,2011,51.0,10.0,6,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100001,11,2011,39.0,50.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p100006,11,2015,18.0,100.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100009,11,2015,52.0,5.0,8,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p10001,11,2014,51.0,100.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,p100010,11,2008,52.0,10.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,p100014,11,2013,51.0,10.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,p10002,11,2015,51.0,200.0,6,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,p100027,11,2014,52.0,10.0,7,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,p100028,11,2006,51.0,300.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Recruit Feature

In [663]:
recruit = pd.read_csv(data_path + '9recruit.csv')

def cleanPNUM(df):
    data = df['PNUM'].fillna('0').values
    pnum = []
    for row in data:
        if '人' in row:
            #print row
            row = int(row[:-1])
        elif row!='若干':
            row = int(row)
        elif row=='若干':
            row = 4.7
        pnum.append(row)

    df['PNUM'] = pnum
    return df


# In[107]:

recruit = cleanPNUM(recruit)

/opt/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [664]:
recruit.RECDATE.unique()

array(['2014-05', '2014-06', '2015-08', '2014-07', '2015-03', '2014-08',
       '2015-04', '2014-09', '2015-07', '2014-10', '2014-11', '2015-06',
       '2014-12', '2015-05', '2015-01', '2015-02', '2014-04', '2014-01',
       '2013-12', '2013-11', '2013-10', '2013-07', '2014-03', '2013-09',
       '2013-08', '2014-02', '2013-04', '2013-06', '2013-05', '2013-03',
       '2008-08', '2012-12', '2012-04'], dtype=object)

In [665]:
recruit.columns

Index(['EID', 'WZCODE', 'POSCODE', 'RECDATE', 'PNUM'], dtype='object')

In [666]:
new_recurit = recruit.groupby(recruit.EID).size().reset_index().rename(columns = {0: 'recurit_count'})

recruit_type = recruit.groupby(['EID', 'WZCODE'])['PNUM'].sum().unstack().add_prefix('WZCODE_').add_suffix('_count').reset_index()
recruit_type.fillna(0, inplace = True)
recurit_2015 = recruit[recruit.RECDATE >= '2015-01']
recurit_2015_count = recurit_2015.groupby(recurit_2015.EID)['PNUM'].sum().reset_index().rename(columns = {0: 'recurit2015_count'})

new_recurit1 = pd.merge(new_recurit,recruit_type , on = 'EID', how = 'left')
new_recurit2 = pd.merge(new_recurit1,recurit_2015_count , on = 'EID', how = 'left')

new_recurit2.fillna(0, inplace = True)

In [667]:
recruit_type

WZCODE,EID,WZCODE_zp01_count,WZCODE_zp02_count,WZCODE_zp03_count
0,p100004,0.0,8.0,24.0
1,p100008,4.0,0.0,23.5
2,p100032,0.0,0.0,6.0
3,p100037,1.0,4.0,22.0
4,p100038,0.0,4.7,10.4
5,p100049,0.0,14.1,0.0
6,p100057,6.0,76.1,7.0
7,p100072,0.0,6.0,0.0
8,p100087,0.0,2.0,14.1
9,p100089,0.0,10.0,0.0


version 2 （有效果）

In [668]:
recruit_type1 = pd.DataFrame(recruit_type['WZCODE_zp01_count'] +  recruit_type['WZCODE_zp02_count'] +  recruit_type['WZCODE_zp03_count'], columns = ['recurit_sum'])

# **
recruit_type1['EID'] = recruit_type.EID
recruit_type1['zp01_rate'] =  recruit_type['WZCODE_zp01_count'] / recruit_type1['recurit_sum']
recruit_type1['zp02_rate'] =  recruit_type['WZCODE_zp02_count'] / recruit_type1['recurit_sum']
recruit_type1['zp03_rate'] =  recruit_type['WZCODE_zp03_count'] / recruit_type1['recurit_sum']
recruit_type1.fillna(0, inplace = True)

new_recurit3 = pd.merge(new_recurit2, recruit_type1 , on = 'EID', how = 'left')

v3.4

In [669]:
"""new_recurit4 = pd.merge(new_recurit3, EID_HY_ETYPE, on = 'EID', how = 'left')

prefix_index = -3
new_recurit5 = get_HandE(new_recurit4, 'recurit_sum', prefix_index)

new_recurit5.drop(['ETYPE', 'HY'], axis = 1, inplace = True)"""

"new_recurit4 = pd.merge(new_recurit3, EID_HY_ETYPE, on = 'EID', how = 'left')\n\nprefix_index = -3\nnew_recurit5 = get_HandE(new_recurit4, 'recurit_sum', prefix_index)\n\nnew_recurit5.drop(['ETYPE', 'HY'], axis = 1, inplace = True)"

v4.4

In [670]:
recurit_newdate = recruit.copy()
recurit_newdate.RECDATE = recurit_newdate.RECDATE.apply(translateYearRECRUIT)


RECDATE_sta = recurit_newdate['RECDATE'].groupby(recurit_newdate.EID).agg([max, min])
RECDATE_sta['range'] = RECDATE_sta['max'] - RECDATE_sta['min']
RECDATE_sta = RECDATE_sta.add_prefix('recruit_RECDATE_').reset_index()

new_recurit4 = pd.merge(new_recurit3, RECDATE_sta , on = 'EID', how = 'left')

Merge Train and Test

In [671]:
recruitDAE = new_recurit4.copy()

In [672]:
train9 = pd.merge(train8, recruitDAE, on = 'EID', how = 'left')
test9 = pd.merge(test8, recruitDAE, on = 'EID', how = 'left')

In [673]:
test9

,EID,PROV,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,WZCODE_zp02_count,WZCODE_zp03_count,PNUM,recurit_sum,zp01_rate,zp02_rate,zp03_rate,recruit_RECDATE_max,recruit_RECDATE_min,recruit_RECDATE_range
0,p10000,11,2011,51.0,10.0,6,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100001,11,2011,39.0,50.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p100006,11,2015,18.0,100.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100009,11,2015,52.0,5.0,8,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p10001,11,2014,51.0,100.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,p100010,11,2008,52.0,10.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,p100014,11,2013,51.0,10.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,p10002,11,2015,51.0,200.0,6,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,p100027,11,2014,52.0,10.0,7,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,p100028,11,2006,51.0,300.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Entbase_Alter Feature

version 3

In [674]:
ent_alter = pd.merge(cdf, entbase, on = 'EID', how ='left')
# HY ALTER 均值与差值

ent_alter_gb_HY = ent_alter.groupby(['HY']).size().reset_index().rename(columns = {0: 'ent_alter_HY_alterno_count'})
HY_EID_count = entbase.groupby(['HY']).size().reset_index().rename(columns = {0: 'entbase_HY_EID_count'})

ent_alter_gb_HY1 = pd.merge(ent_alter_gb_HY, HY_EID_count, on = 'HY', how = 'left')
ent_alter_gb_HY1['ent_alter_HY_alterno_mean'] = ent_alter_gb_HY1['ent_alter_HY_alterno_count'] / ent_alter_gb_HY1['entbase_HY_EID_count']
ent_alter_gb_HY2 = ent_alter_gb_HY1.drop(['ent_alter_HY_alterno_count' , 'entbase_HY_EID_count'], axis = 1)


# ETYPE ALTER 均值与差值
ent_alter_gb_ETYPE = ent_alter.groupby(['ETYPE']).size().reset_index().rename(columns = {0: 'ent_alter_ETYPE_alterno_count'})
ETYPE_EID_count = entbase.groupby(['ETYPE']).size().reset_index().rename(columns = {0: 'entbase_ETYPE_EID_count'})

ent_alter_gb_ETYPE1 = pd.merge(ent_alter_gb_ETYPE, ETYPE_EID_count, on = 'ETYPE', how = 'left')
ent_alter_gb_ETYPE1['ent_alter_ETYPE_alterno_mean'] = ent_alter_gb_ETYPE1['ent_alter_ETYPE_alterno_count'] / ent_alter_gb_ETYPE1['entbase_ETYPE_EID_count']
ent_alter_gb_ETYPE2 = ent_alter_gb_ETYPE1.drop(['ent_alter_ETYPE_alterno_count' , 'entbase_ETYPE_EID_count'], axis = 1)

ent_alter_gb_HY_ETYPE = ent_alter.groupby(['ETYPE', 'HY']).size().reset_index().rename(columns = {0: 'ent_alter_ETYPE_alterno_count'})
ETYPE_EID_count = entbase.groupby(['ETYPE']).size().reset_index().rename(columns = {0: 'entbase_ETYPE_EID_count'})

ent_alter_gb_ETYPE1 = pd.merge(ent_alter_gb_ETYPE, ETYPE_EID_count, on = 'ETYPE', how = 'left')
ent_alter_gb_ETYPE1['ent_alter_ETYPE_alterno_mean'] = ent_alter_gb_ETYPE1['ent_alter_ETYPE_alterno_count'] / ent_alter_gb_ETYPE1['entbase_ETYPE_EID_count']
ent_alter_gb_ETYPE2 = ent_alter_gb_ETYPE1.drop(['ent_alter_ETYPE_alterno_count' , 'entbase_ETYPE_EID_count'], axis = 1)

In [675]:
train10 = pd.merge(train9, ent_alter_gb_HY2, on = 'HY', how = 'left')
train11 = pd.merge(train10, ent_alter_gb_ETYPE2, on = 'ETYPE', how = 'left')
train11.ent_alter_ETYPE_alterno_mean.fillna(0, inplace = True)
train11.ent_alter_HY_alterno_mean.fillna(0, inplace = True)
train11['ent_alter_HY_alterno_gap'] = train11['ALT_NUM'] - train11['ent_alter_HY_alterno_mean']
train11['ent_alter_ETYPE_alterno_gap'] = train11['ALT_NUM'] - train11['ent_alter_ETYPE_alterno_mean']

test10 = pd.merge(test9, ent_alter_gb_HY2, on = 'HY', how = 'left')
test11= pd.merge(test10, ent_alter_gb_ETYPE2, on = 'ETYPE', how = 'left')
test11.ent_alter_ETYPE_alterno_mean.fillna(0, inplace = True)
test11.ent_alter_HY_alterno_mean.fillna(0, inplace = True)
test11['ent_alter_HY_alterno_gap'] = test11['ALT_NUM'] - test11['ent_alter_HY_alterno_mean']
test11['ent_alter_ETYPE_alterno_gap'] = test11['ALT_NUM'] - test11['ent_alter_ETYPE_alterno_mean']

# Qualification

In [676]:
qualification = pd.read_csv(data_path + '10qualification.csv')

In [677]:
def translateYearQualification(date):
    if type(date) == float:
        return np.nan
    year = int(date[:4])
    month = int(date[-2:])
    return (year-2007)*12 + month

In [678]:
qualification_count = qualification.groupby(qualification.EID).size().reset_index().rename(columns = {0: 'qualification_count'})

new_qualification = qualification.copy()
new_qualification['BEGINDATE'] = new_qualification.BEGINDATE.apply(translateYearQualification)
new_qualification['EXPIRYDATE'] = new_qualification.EXPIRYDATE.apply(translateYearQualification)

qualification_type = new_qualification.groupby(['EID', 'ADDTYPE']).size().unstack().add_prefix('qualification_type_').add_suffix('_count').reset_index()
qualification_type.fillna(0, inplace = True)

qualification_bdate_sta = new_qualification.groupby(new_qualification.EID).BEGINDATE.agg([max, min]).add_prefix('Qua_Bdate_').reset_index()
qualification_bdate_sta['Qua_Bdate_range'] = qualification_bdate_sta['Qua_Bdate_max'] - qualification_bdate_sta['Qua_Bdate_min']
qualification_edate_sta = new_qualification.groupby(new_qualification.EID).EXPIRYDATE.agg([max, min]).add_prefix('Qua_Edate_').reset_index()

new_qualification2 = pd.merge(qualification_count, qualification_type, on = 'EID', how = 'left')
new_qualification3 = pd.merge(new_qualification2, qualification_bdate_sta, on = 'EID', how = 'left')
new_qualification4 = pd.merge(new_qualification3, qualification_edate_sta, on = 'EID', how = 'left')

In [679]:
qualificationDAE = new_qualification4.copy()

In [680]:
train12 = pd.merge(train11, qualificationDAE, on = 'EID', how = 'left')
test12 = pd.merge(test11, qualificationDAE, on = 'EID', how = 'left')

In [681]:
import copy
def get_nan_info(nan_train):
    tmp = copy.deepcopy(nan_train)
    tmp.fillna(-999, inplace = True)
    tmp['n_null'] = (tmp == -999).sum(axis=1)
    tmp['discret_null'] = tmp.n_null
    tmp.discret_null[tmp.discret_null<=40] = 1
    tmp.discret_null[(tmp.discret_null>40)&(tmp.discret_null<=73)] = 2
    tmp.discret_null[(tmp.discret_null>73)&(tmp.discret_null<=82)] = 3
    tmp.discret_null[(tmp.discret_null>82)&(tmp.discret_null<=85)] = 4
    tmp.discret_null[(tmp.discret_null>85)&(tmp.discret_null<=91)] = 5
    tmp.discret_null[(tmp.discret_null>91)&(tmp.discret_null<=100)] = 6
    tmp.discret_null[tmp.discret_null>100] = 7
    return tmp


In [682]:
temp = copy.deepcopy(train12)
temp1 = get_nan_info(temp)
train13 = pd.concat([train12, temp1[['discret_null']] ], axis = 1)

temp2 = copy.deepcopy(test12)
temp3 = get_nan_info(temp2)
test13 = pd.concat([test12, temp3[['discret_null']] ], axis = 1)

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying t

# Save Feature (Remember change the file name)

In [683]:
#train13.drop(['ENDDATE_n'], axis = 1, inplace = True)

In [684]:
train13['len_EID_NUMBER'] = train13['EID_NUMBER'].apply(lambda x:int(x/20))
train13 = train13.drop(['EID_NUMBER'], axis=1)
test13['len_EID_NUMBER'] = test13['EID_NUMBER'].apply(lambda x:int(x/20))
test13 = test13.drop(['EID_NUMBER'], axis=1)

savetrain = train13.copy()
savetest = test13.copy()

In [ ]:
binlabel_train_cp = pd.read_csv(feature_path + 'binlabel_train_cp.csv')
binlabel_test_cp = pd.read_csv(feature_path + 'binlabel_test_cp.csv')


#bin_feature = pd.read_csv(feature_path + 't30_bin_feature.csv')

bin_label_train = pd.merge(savetrain, binlabel_train_cp, on = 'EID', how = 'left')
bin_label_test = pd.merge(savetest, binlabel_test_cp, on = 'EID', how = 'left')

bin_label_train.shape , bin_label_test.shape

bin_label_train.to_csv(feature_path + 'train_v1.8.csv', index = False)
bin_label_test.to_csv(feature_path + 'test_v1.8.csv', index = False)

In [696]:
#savetrain.to_csv(feature_path + 'train_v1.2.csv', index = False)
#savetest.to_csv(feature_path + 'test_v1.2.csv', index = False)